In [ ]:
!pip install torch
!pip install opencv-python
!pip install numpy
!pip install Pillow
!pip install scipy

In [4]:
import torch
import cv2
import numpy as np
from PIL import Image
import time
import scipy
from scipy.spatial import distance

In [11]:
prev_time = 0

#yolov5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weight.pt')

cap = cv2.VideoCapture("x.mp4")
cap.set(3, 1280)
cap.set(4, 720)

dt = 1/30.0
kf = cv2.KalmanFilter(6, 2)
kf.measurementMatrix = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]], np.float32)  
kf.transitionMatrix = np.array([[1, 0, 0, dt, 0, 0], [0, 1, 0, 0, dt, 0], [0, 0, 1, 0, 0, dt],
                                [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]], np.float32)  
kf.processNoiseCov = 1e-5 * np.eye(6, dtype=np.float32)  
kf.measurementNoiseCov = 1e-3 * np.eye(2, dtype=np.float32) 

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
record = cv2.VideoWriter('record.mp4', fourcc, 5, (960,5))

class CentroidTracker:
    def __init__(self, maxDisappeared=50):
        self.nextObjectID = 0
        self.objects = {}
        self.disappeared = {}
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")

        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            D = scipy.spatial.distance.cdist(np.array(objectCentroids), inputCentroids)

            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                usedRows.add(row)
                usedCols.add(col)

            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        return self.objects

centroid_tracker = CentroidTracker()

# Variables for the timer
timer_start = None
inside_time = 0
lock_duration = 5  # Locking time (seconds)
is_inside = False

while True:
    new_frame = time.time()

    ret, frame = cap.read()

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = model(Image.fromarray(img))

    rects = []
    for obj_temp in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = obj_temp.tolist()
        if conf > 0.5:
            obj = np.array([[(x1 + x2) / 2], [(y1 + y2) / 2], [0], [0], [0], [0]], dtype=np.float32)
            prediction = kf.predict()
            kf.correct(obj[:2])
            x, y, z, dx, dy, dz = prediction.ravel()
            x, y = int(x), int(y)
            rects.append((x - 10, y - 10, x + 10, y + 10))

    if rects:  # continue if rects are not empty
        objects = centroid_tracker.update(rects)

        for (objectID, centroid) in objects.items():
            x, y = centroid
            cv2.rectangle(frame, (x - 10, y - 10), (x + 10, y + 10), (0, 255, 0), 2)
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

        # Check if it is within the specified rectangle
        if int(img.shape[1] / 4) < x < int(3 * img.shape[1] / 4) and int(img.shape[0] / 10) < y < int(9 * img.shape[0] / 10):
            if not is_inside:
                timer_start = time.time()
                is_inside = True
        else:
            if is_inside:
                is_inside = False
                timer_start = None
                inside_time = 0

        # Timer operations
        if timer_start:
            elapsed_time = time.time() - timer_start
            inside_time = elapsed_time

            if elapsed_time > lock_duration:
                cv2.putText(frame, f"Lock Successful! ({elapsed_time:.2f} seconds)", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, f"Locking... ({elapsed_time:.2f} seconds)", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Draw the line
        for i in range(1, len(objects)):
            if list(objects.values())[i - 1] is not None and list(objects.values())[i] is not None:
                pts = np.array([list(objects.values())[i - 1], list(objects.values())[i]], np.int32)
                pts = pts.reshape((-1, 1, 2))
                cv2.polylines(frame, [pts], False, (0, 255, 0), 2)

    x1 = int(0.25 * cap.get(3))  
    y1 = int(0.1 * cap.get(4))  
    x2 = int(0.75 * cap.get(3))  
    y2 = int(0.9 * cap.get(4))  
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    for obj_temp in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = obj_temp.tolist()
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        if conf > 0.5:
            label = f'{model.names[int(cls)]} {conf:.2f}'
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    curr_time = time.time()
    fps = 1 / (curr_time - prev_time)
    prev_time = curr_time
    cv2.putText(frame, f'FPS: {fps:.2f}', (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    cv2.imshow("detect", frame)

    record.write(frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
record.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\batikan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-1-24 Python-3.11.5 torch-2.1.2+cpu CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 
